In [1]:
import pandas as pd

In [2]:
"""
Loads in raw score file
Expecting the schema 
Name	Date	GF	GA	Team

Calcuates result, margies and goal difference
Does some prep work for Form and Win

Obviously names need to be consistant
"""

raw_data = pd.read_csv("../data/results_v2.csv")
raw_data["GD"] = raw_data["GF"] - raw_data["GA"]
raw_data.loc[(raw_data['GD'] == 0), 'GamePoints'] = 0.5
raw_data.loc[(raw_data['GD'] < 0), 'GamePoints'] = -1
raw_data.loc[(raw_data['GD'] > 0), 'GamePoints'] = 1
raw_data.loc[(raw_data['GD'] > 6), 'GamePoints'] = 1.5
raw_data.loc[(raw_data['GD'] < -6), 'GamePoints'] = -1.5
raw_data.loc[(raw_data['GamePoints'] >= 1), 'W'] = 1
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'D'] = 1
raw_data.loc[(raw_data['GamePoints'] <= -1), 'L'] = 1
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'MW'] = 1
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'ML'] = 1

raw_data.loc[(raw_data['GamePoints'] >= 1), 'Result'] = '🔼'
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'Result'] = '➖'
raw_data.loc[(raw_data['GamePoints'] <= -1), 'Result'] = '🔽'
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'Result'] = '⏫'
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'Result'] = '⏬'

raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers

raw_data = raw_data.fillna(0)
pd.set_option('display.float_format', '{:.1f}'.format)
raw_data = raw_data.astype({'W':'int','D':'int','L':'int','MW':'int','ML':'int'})

raw_data['Date'] = pd.to_datetime(raw_data['Date'],dayfirst = True)
gameweek_int = raw_data.nunique().Date
gameweek = "Gameweek " + str(gameweek_int)

#display(raw_data)

In [3]:
"""
Creates the Form Column
"""
def form_column(raw_data):
    raw_data['game_rank'] = raw_data.sort_values("Date").groupby("Name").Date.rank(ascending=False)
    raw_data_form = raw_data.loc[raw_data['game_rank'] <= 5]
    form = raw_data_form.groupby(['Name'], as_index=False).agg({'Result': ' '.join})
    return form

form = form_column(raw_data)

#display(form)

In [4]:
"""
Creates Matches Played (MP), Partipation (Par%), Pts and Win% (W%)

It also aggreates all the raw data to create an up to date table
"""
def get_totals(raw_data):
    played = raw_data.groupby('Name').count()
    played = pd.DataFrame(played["Date"]).rename(columns={"Date":"MP"})
    data = raw_data.groupby('Name').sum(numeric_only=True).sort_values(by=['GamePoints'],ascending=False).join(played).rename(columns={"GamePoints":"Pts"})

    data["Par%"] = data["MP"].div(gameweek_int)*100
    data["W%"] = data["W"]/data["MP"]*100
    data = data.astype({'Par%':'int','W%':'int'})
    return data

totals = get_totals(raw_data)



#display(data)


In [5]:
"""
Join tables 
Drops unwanted columns 
Puts columns in the correct order
"""
table = totals.merge(form,on='Name').rename(columns={"Result":"Form"})
table.index = table.Name
table = table[['MP', 'W', 'D', 'L','MW', 'ML', 'GF', 'GA','GD','Par%','W%','Pts','Form']]
table.index.names = [gameweek]
table

,MP,W,D,L,MW,ML,GF,GA,GD,Par%,W%,Pts,Form
Gameweek 23,,,,,,,,,,,,,
Calz,16,10,1,5,5,1,255,205,50,69,62,7.5,🔼 🔽 ⏫ ➖ 🔼
Schurrle,13,8,0,5,3,2,206,191,15,56,61,3.5,⏬ 🔽 🔼 🔼 🔽
Virlo,9,5,1,3,2,2,141,140,1,39,55,2.5,⏬ 🔼 🔼 ⏬ ➖
Zola,14,7,1,6,3,2,201,207,-6,60,50,2.0,🔼 🔽 ⏬ ➖ 🔽
Gazprom,16,9,0,7,2,3,226,228,-2,69,56,1.5,🔽 🔽 🔼 🔼 🔽
Reaper,19,11,0,8,2,5,265,274,-9,82,57,1.5,⏫ 🔼 ⏫ 🔼 🔼
Keysey,9,4,1,4,1,1,128,117,11,39,44,0.5,⏬ ⏫ ➖ 🔼 🔽
Helmet,18,9,0,9,3,4,249,272,-23,78,50,-0.5,🔼 🔽 ⏬ 🔽 🔼
Peggy,5,2,0,3,1,0,78,70,8,21,40,-0.5,🔽 🔽 🔼 ⏫ 🔽


In [6]:
"""
TO DO

Add Graphics
Tracking player pairings
Tracking week by week table position

"""


'\nTO DO\n\nAdd Graphics\nTracking player pairings\nTracking week by week table position\n\n'

In [7]:
display(raw_data)

,Name,Date,GF,GA,Team,GD,GamePoints,W,D,L,MW,ML,Result,game_rank
0,Calz,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,16.0
1,Helmet,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,18.0
2,Kim,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,19.0
3,Mate,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,20.0
4,Peggy,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,Zola,2023-07-27,11,12,BLANCOS,-1,-1.0,0,0,1,0,0,🔽,1.0
226,Lou,2023-07-27,11,12,BLANCOS,-1,-1.0,0,0,1,0,0,🔽,1.0
227,Keysey,2023-07-27,11,12,BLANCOS,-1,-1.0,0,0,1,0,0,🔽,1.0
228,Schurrle,2023-07-27,11,12,BLANCOS,-1,-1.0,0,0,1,0,0,🔽,1.0


In [8]:
def count_same_team(df, name_col, date_col, team_col):
    count = 0
    for idx, row in df.iterrows():
        name = row[name_col]
        date = row[date_col]
        team = row[team_col]
        same_team_df = df[(df[team_col] == team) & (df[date_col] == date)]
        same_team_names = same_team_df[name_col].unique()
        if len(same_team_names) > 1 and name in same_team_names:
            print(same_team_names)
    return count

In [9]:
count_same_team(raw_data, 'Name', 'Date', 'Team')

['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Lou']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Lou']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Lou']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Lou']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Lou']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']

0